In [1]:
from pulp import *
import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

## Read data source

Read individual attributes and convert it to a matrix

In [2]:
fields = ['Pay']
df1 = pd.read_excel(r'C:\Users\jthekkel\Documents\Optimization\NurseOptimizationDetail.xlsx',sheet_name='Data',usecols=fields)
np_pay = df1.as_matrix()
#print(df1)


fields = ['Quality']
df2 = pd.read_excel(r'C:\Users\jthekkel\Documents\Optimization\NurseOptimizationDetail.xlsx',sheet_name='Data',usecols=fields)
np_qual = df2.as_matrix()
#print(df2)


C:\Users\jthekkel\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jthekkel\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


Read final attribute table

In [3]:
fields = ['MondayAvail','TuesdayAvail','WednesdayAvail','ThursdayAvail','FridayAvail']
dfAvail = pd.read_excel(r'C:\Users\jthekkel\Documents\Optimization\NurseOptimizationDetail.xlsx',sheet_name='Data',usecols=fields)
dfAvail.replace(0,np.nan, inplace = True)

print(dfAvail)

    MondayAvail  TuesdayAvail  WednesdayAvail  ThursdayAvail  FridayAvail
0           1.0           1.0             1.0            1.0          NaN
1           1.0           1.0             1.0            1.0          1.0
2           1.0           1.0             NaN            NaN          NaN
3           1.0           NaN             1.0            NaN          1.0
4           NaN           1.0             NaN            1.0          1.0
5           NaN           1.0             NaN            NaN          1.0
6           1.0           NaN             1.0            NaN          1.0
7           1.0           1.0             NaN            NaN          NaN
8           NaN           1.0             1.0            1.0          1.0
9           1.0           1.0             NaN            1.0          1.0
10          NaN           1.0             1.0            1.0          1.0
11          1.0           NaN             1.0            NaN          1.0
12          1.0           1.0         

Matrix multuplication - multiply all attributes

In [4]:
df = dfAvail.multiply(np_pay,axis=0)
df = df.multiply(np_qual,axis=0)
print(df)

    MondayAvail  TuesdayAvail  WednesdayAvail  ThursdayAvail  FridayAvail
0          18.0          18.0            18.0           18.0          NaN
1         180.0         180.0           180.0          180.0        180.0
2          95.0          95.0             NaN            NaN          NaN
3          20.0           NaN            20.0            NaN         20.0
4           NaN          20.0             NaN           20.0         20.0
5           NaN          20.0             NaN            NaN         20.0
6          40.0           NaN            40.0            NaN         40.0
7          60.0          60.0             NaN            NaN          NaN
8           NaN         140.0           140.0          140.0        140.0
9         200.0         200.0             NaN          200.0        200.0
10          NaN          63.0            63.0           63.0         63.0
11         63.0           NaN            63.0            NaN         63.0
12        132.0         132.0         

## Create Problem  

In [5]:
#We want to minimize the cost
prob = pulp.LpProblem('ScheduleNurse',pulp.LpMinimize)

## Decision Variables 

In [6]:
dfAvail['ID'] = dfAvail.reset_index().index

RM = dfAvail.ID.as_matrix()

dfAvail = dfAvail.multiply(RM,axis=0)
dfAvail = dfAvail.drop(columns=['ID'])

print RM
print(dfAvail)	


[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
    MondayAvail  TuesdayAvail  WednesdayAvail  ThursdayAvail  FridayAvail
0           0.0           0.0             0.0            0.0          NaN
1           1.0           1.0             1.0            1.0          1.0
2           2.0           2.0             NaN            NaN          NaN
3           3.0           NaN             3.0            NaN          3.0
4           NaN           4.0             NaN            4.0          4.0
5           NaN           5.0             NaN            NaN          5.0
6           6.0           NaN             6.0            NaN          6.0
7           7.0           7.0             NaN            NaN          NaN
8           NaN           8.0             8.0            8.0          8.0
9           9.0           9.0             NaN            9.0          9.0
10          NaN          10.0            10.0           10.0         10.0
11         11.0           NaN            11.0            NaN         11

C:\Users\jthekkel\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [7]:
released = {
		"MondayAvail" : 0,
		"TuesdayAvail" : 1,
		"WednesdayAvail" : 2,
		"ThursdayAvail" : 3,
		"FridayAvail" : 4
	}
print released

print released.get("MondayAvail", "none")

{'TuesdayAvail': 1, 'FridayAvail': 4, 'ThursdayAvail': 3, 'MondayAvail': 0, 'WednesdayAvail': 2}
0


In [8]:
colname =""
day = ""
list2 =[]
n = 0


for col in dfAvail:
    for i,d in dfAvail.iteritems():
        colname = i
        for index, rows in dfAvail.iterrows():
            if colname == col:
                day = released.get(col, "none")
                my_list = [colname,rows[day]]
                list2.append(my_list)

             
print list2


[['MondayAvail', 0.0], ['MondayAvail', 1.0], ['MondayAvail', 2.0], ['MondayAvail', 3.0], ['MondayAvail', nan], ['MondayAvail', nan], ['MondayAvail', 6.0], ['MondayAvail', 7.0], ['MondayAvail', nan], ['MondayAvail', 9.0], ['MondayAvail', nan], ['MondayAvail', 11.0], ['MondayAvail', 12.0], ['TuesdayAvail', 0.0], ['TuesdayAvail', 1.0], ['TuesdayAvail', 2.0], ['TuesdayAvail', nan], ['TuesdayAvail', 4.0], ['TuesdayAvail', 5.0], ['TuesdayAvail', nan], ['TuesdayAvail', 7.0], ['TuesdayAvail', 8.0], ['TuesdayAvail', 9.0], ['TuesdayAvail', 10.0], ['TuesdayAvail', nan], ['TuesdayAvail', 12.0], ['WednesdayAvail', 0.0], ['WednesdayAvail', 1.0], ['WednesdayAvail', nan], ['WednesdayAvail', 3.0], ['WednesdayAvail', nan], ['WednesdayAvail', nan], ['WednesdayAvail', 6.0], ['WednesdayAvail', nan], ['WednesdayAvail', 8.0], ['WednesdayAvail', nan], ['WednesdayAvail', 10.0], ['WednesdayAvail', 11.0], ['WednesdayAvail', nan], ['ThursdayAvail', 0.0], ['ThursdayAvail', 1.0], ['ThursdayAvail', nan], ['ThursdayA

In [9]:
list3 = []
dv_list = []

for i,r in list2:
    #if r > 0:
    list3 = [i,r]
    dv_list.append(list3)

print(dv_list)
        

[['MondayAvail', 0.0], ['MondayAvail', 1.0], ['MondayAvail', 2.0], ['MondayAvail', 3.0], ['MondayAvail', nan], ['MondayAvail', nan], ['MondayAvail', 6.0], ['MondayAvail', 7.0], ['MondayAvail', nan], ['MondayAvail', 9.0], ['MondayAvail', nan], ['MondayAvail', 11.0], ['MondayAvail', 12.0], ['TuesdayAvail', 0.0], ['TuesdayAvail', 1.0], ['TuesdayAvail', 2.0], ['TuesdayAvail', nan], ['TuesdayAvail', 4.0], ['TuesdayAvail', 5.0], ['TuesdayAvail', nan], ['TuesdayAvail', 7.0], ['TuesdayAvail', 8.0], ['TuesdayAvail', 9.0], ['TuesdayAvail', 10.0], ['TuesdayAvail', nan], ['TuesdayAvail', 12.0], ['WednesdayAvail', 0.0], ['WednesdayAvail', 1.0], ['WednesdayAvail', nan], ['WednesdayAvail', 3.0], ['WednesdayAvail', nan], ['WednesdayAvail', nan], ['WednesdayAvail', 6.0], ['WednesdayAvail', nan], ['WednesdayAvail', 8.0], ['WednesdayAvail', nan], ['WednesdayAvail', 10.0], ['WednesdayAvail', 11.0], ['WednesdayAvail', nan], ['ThursdayAvail', 0.0], ['ThursdayAvail', 1.0], ['ThursdayAvail', nan], ['ThursdayA

In [10]:
dv=[]

#dv = set(tuple(element) for element in list2)
dv = sorted(dv)
print dv

[]


In [11]:
print sorted(dv)

[]


In [12]:
decision_variables =[]

for l,r in dv_list:
    if r >= 0:
        variable = str('x' + str(r) + str(l) )
        variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
        decision_variables.append(variable)
    
print(str(decision_variables))
print(str(len(decision_variables)))    
    

[x0.0MondayAvail, x1.0MondayAvail, x2.0MondayAvail, x3.0MondayAvail, x6.0MondayAvail, x7.0MondayAvail, x9.0MondayAvail, x11.0MondayAvail, x12.0MondayAvail, x0.0TuesdayAvail, x1.0TuesdayAvail, x2.0TuesdayAvail, x4.0TuesdayAvail, x5.0TuesdayAvail, x7.0TuesdayAvail, x8.0TuesdayAvail, x9.0TuesdayAvail, x10.0TuesdayAvail, x12.0TuesdayAvail, x0.0WednesdayAvail, x1.0WednesdayAvail, x3.0WednesdayAvail, x6.0WednesdayAvail, x8.0WednesdayAvail, x10.0WednesdayAvail, x11.0WednesdayAvail, x0.0ThursdayAvail, x1.0ThursdayAvail, x4.0ThursdayAvail, x8.0ThursdayAvail, x9.0ThursdayAvail, x10.0ThursdayAvail, x12.0ThursdayAvail, x1.0FridayAvail, x3.0FridayAvail, x4.0FridayAvail, x5.0FridayAvail, x6.0FridayAvail, x8.0FridayAvail, x9.0FridayAvail, x10.0FridayAvail, x11.0FridayAvail, x12.0FridayAvail]
43


## Objective Function 

In [13]:
Mon=[]
Tue=[]
Wed=[]
Thu=[]
Fri=[]

for i, r in df.iterrows():
    Mon.append(r[0])
    Tue.append(r[1])
    Wed.append(r[2])
    Thu.append(r[3])
    Fri.append(r[4])
    
Scorenp = np.concatenate((Mon,Tue,Wed,Thu,Fri), axis=0)  
#print Scorenp
columns = ['Score']
Score = pd.DataFrame(Scorenp, columns=columns)
#print(Score)

In [14]:
print(Scorenp)

[ 18. 180.  95.  20.  nan  nan  40.  60.  nan 200.  nan  63. 132.  18.
 180.  95.  nan  20.  20.  nan  60. 140. 200.  63.  nan 132.  18. 180.
  nan  20.  nan  nan  40.  nan 140.  nan  63.  63.  nan  18. 180.  nan
  nan  20.  nan  nan  nan 140. 200.  63.  nan 132.  nan 180.  nan  20.
  20.  20.  40.  nan 140. 200.  63.  63. 132.]


In [15]:
for i, s in enumerate(decision_variables):
    print i,s

0 x0.0MondayAvail
1 x1.0MondayAvail
2 x2.0MondayAvail
3 x3.0MondayAvail
4 x6.0MondayAvail
5 x7.0MondayAvail
6 x9.0MondayAvail
7 x11.0MondayAvail
8 x12.0MondayAvail
9 x0.0TuesdayAvail
10 x1.0TuesdayAvail
11 x2.0TuesdayAvail
12 x4.0TuesdayAvail
13 x5.0TuesdayAvail
14 x7.0TuesdayAvail
15 x8.0TuesdayAvail
16 x9.0TuesdayAvail
17 x10.0TuesdayAvail
18 x12.0TuesdayAvail
19 x0.0WednesdayAvail
20 x1.0WednesdayAvail
21 x3.0WednesdayAvail
22 x6.0WednesdayAvail
23 x8.0WednesdayAvail
24 x10.0WednesdayAvail
25 x11.0WednesdayAvail
26 x0.0ThursdayAvail
27 x1.0ThursdayAvail
28 x4.0ThursdayAvail
29 x8.0ThursdayAvail
30 x9.0ThursdayAvail
31 x10.0ThursdayAvail
32 x12.0ThursdayAvail
33 x1.0FridayAvail
34 x3.0FridayAvail
35 x4.0FridayAvail
36 x5.0FridayAvail
37 x6.0FridayAvail
38 x8.0FridayAvail
39 x9.0FridayAvail
40 x10.0FridayAvail
41 x11.0FridayAvail
42 x12.0FridayAvail


In [16]:
Scorenp = [Scorenp for Scorenp in Scorenp if str(Scorenp) != 'nan']
print (Scorenp)

[18.0, 180.0, 95.0, 20.0, 40.0, 60.0, 200.0, 63.0, 132.0, 18.0, 180.0, 95.0, 20.0, 20.0, 60.0, 140.0, 200.0, 63.0, 132.0, 18.0, 180.0, 20.0, 40.0, 140.0, 63.0, 63.0, 18.0, 180.0, 20.0, 140.0, 200.0, 63.0, 132.0, 180.0, 20.0, 20.0, 20.0, 40.0, 140.0, 200.0, 63.0, 63.0, 132.0]


In [17]:
total_cost = ""
for j, row in enumerate(Scorenp):
    for i, schedule in enumerate(decision_variables):
        if j == i:
            #if row > 0:
            formula = row * schedule
            total_cost += formula
            #print(j,i)
            
#print(total_cost)

prob += total_cost
print ("Optimization function: " + str(total_cost))

Optimization function: 18.0*x0.0MondayAvail + 18.0*x0.0ThursdayAvail + 18.0*x0.0TuesdayAvail + 18.0*x0.0WednesdayAvail + 180.0*x1.0FridayAvail + 180.0*x1.0MondayAvail + 180.0*x1.0ThursdayAvail + 180.0*x1.0TuesdayAvail + 180.0*x1.0WednesdayAvail + 63.0*x10.0FridayAvail + 63.0*x10.0ThursdayAvail + 63.0*x10.0TuesdayAvail + 63.0*x10.0WednesdayAvail + 63.0*x11.0FridayAvail + 63.0*x11.0MondayAvail + 63.0*x11.0WednesdayAvail + 132.0*x12.0FridayAvail + 132.0*x12.0MondayAvail + 132.0*x12.0ThursdayAvail + 132.0*x12.0TuesdayAvail + 95.0*x2.0MondayAvail + 95.0*x2.0TuesdayAvail + 20.0*x3.0FridayAvail + 20.0*x3.0MondayAvail + 20.0*x3.0WednesdayAvail + 20.0*x4.0FridayAvail + 20.0*x4.0ThursdayAvail + 20.0*x4.0TuesdayAvail + 20.0*x5.0FridayAvail + 20.0*x5.0TuesdayAvail + 40.0*x6.0FridayAvail + 40.0*x6.0MondayAvail + 40.0*x6.0WednesdayAvail + 60.0*x7.0MondayAvail + 60.0*x7.0TuesdayAvail + 140.0*x8.0FridayAvail + 140.0*x8.0ThursdayAvail + 140.0*x8.0TuesdayAvail + 140.0*x8.0WednesdayAvail + 200.0*x9.0Frid

for j, row in enumerate(Scorenp):
    print row

## Constraint 

In [18]:
for i, schedule in enumerate(decision_variables[0:]):
    print schedule

x0.0MondayAvail
x1.0MondayAvail
x2.0MondayAvail
x3.0MondayAvail
x6.0MondayAvail
x7.0MondayAvail
x9.0MondayAvail
x11.0MondayAvail
x12.0MondayAvail
x0.0TuesdayAvail
x1.0TuesdayAvail
x2.0TuesdayAvail
x4.0TuesdayAvail
x5.0TuesdayAvail
x7.0TuesdayAvail
x8.0TuesdayAvail
x9.0TuesdayAvail
x10.0TuesdayAvail
x12.0TuesdayAvail
x0.0WednesdayAvail
x1.0WednesdayAvail
x3.0WednesdayAvail
x6.0WednesdayAvail
x8.0WednesdayAvail
x10.0WednesdayAvail
x11.0WednesdayAvail
x0.0ThursdayAvail
x1.0ThursdayAvail
x4.0ThursdayAvail
x8.0ThursdayAvail
x9.0ThursdayAvail
x10.0ThursdayAvail
x12.0ThursdayAvail
x1.0FridayAvail
x3.0FridayAvail
x4.0FridayAvail
x5.0FridayAvail
x6.0FridayAvail
x8.0FridayAvail
x9.0FridayAvail
x10.0FridayAvail
x11.0FridayAvail
x12.0FridayAvail


In [19]:
Nurse0 = ""
Nurse1 = ""
Nurse2 = ""
Nurse3 = ""
Nurse4 = ""
Nurse5 = ""
Nurse6 = ""
Nurse7 = ""
Nurse8 = ""
Nurse9 = ""
Nurse10 = ""
Nurse11 = ""
Nurse12 = ""

x0 = []
x1 = []
x2 = []
x3 = []
x4 = []
x5 = []
x6 = []
x7 = []
x8 = []
x9 = []
x10 = []
x11 = []
x12 = []

# for X0
for i, schedule in enumerate(decision_variables[0:1]):
    x0.append(schedule)
for i, schedule in enumerate(decision_variables[9:10]):
    x0.append(schedule)
for i, schedule in enumerate(decision_variables[19:20]):
    x0.append(schedule)   
for i, schedule in enumerate(decision_variables[26:27]):
    x0.append(schedule)       
    
for i in x0:
    Nurse0 += i
prob += (Nurse0 <= 2)    
    
    
#for X1
for i, schedule in enumerate(decision_variables[1:2]):
    x1.append(schedule)
for i, schedule in enumerate(decision_variables[10:11]):
    x1.append(schedule)
for i, schedule in enumerate(decision_variables[20:21]):
    x1.append(schedule)    
for i, schedule in enumerate(decision_variables[33:34]):
    x1.append(schedule)     
    
for i in x1:
    Nurse1 += i
prob += (Nurse1 <= 2)    
    

# for X2     
for i, schedule in enumerate(decision_variables[2:3]):
    x2.append(schedule)
for i, schedule in enumerate(decision_variables[11:12]):
    x2.append(schedule)


for i in x2:
    Nurse2 += i
prob += (Nurse2 <= 2)    
    
    
#for X3
for i, schedule in enumerate(decision_variables[3:4]):
    x3.append(schedule)
for i, schedule in enumerate(decision_variables[21:22]):
    x3.append(schedule)
for i, schedule in enumerate(decision_variables[34:35]):
    x3.append(schedule)
    
for i in x3:
    Nurse3 += i
prob += (Nurse3 <= 2)      


#for X4
for i, schedule in enumerate(decision_variables[12:13]):
    x4.append(schedule)
for i, schedule in enumerate(decision_variables[28:29]):
    x4.append(schedule)
for i, schedule in enumerate(decision_variables[35:36]):
    x4.append(schedule)    

for i in x4:
    Nurse4 += i
prob += (Nurse4 <= 2)          
    

#for X5
for i, schedule in enumerate(decision_variables[13:14]):
    x5.append(schedule)
for i, schedule in enumerate(decision_variables[36:37]):
    x5.append(schedule) 
    
for i in x5:
    Nurse5 += i
prob += (Nurse5 <= 2)        


#for X6
for i, schedule in enumerate(decision_variables[4:5]):
    x6.append(schedule)
for i, schedule in enumerate(decision_variables[22:23]):
    x6.append(schedule)
for i, schedule in enumerate(decision_variables[37:38]):
    x6.append(schedule)     

for i in x6:
    Nurse6 += i
prob += (Nurse6 <= 2)        
 
    
#for X7
for i, schedule in enumerate(decision_variables[5:6]):
    x7.append(schedule)
for i, schedule in enumerate(decision_variables[14:15]):
    x7.append(schedule)      
    
for i in x7:
    Nurse7 += i
prob += (Nurse7 <= 2)        


#for X8
for i, schedule in enumerate(decision_variables[15:16]):
    x8.append(schedule)
for i, schedule in enumerate(decision_variables[23:24]):
    x8.append(schedule)
for i, schedule in enumerate(decision_variables[29:30]):
    x8.append(schedule)
for i, schedule in enumerate(decision_variables[38:39]):
    x8.append(schedule)    

for i in x8:
    Nurse8 += i
prob += (Nurse8 <= 2)        
      
#for X9
for i, schedule in enumerate(decision_variables[6:7]):
    x9.append(schedule)
for i, schedule in enumerate(decision_variables[16:17]):
    x9.append(schedule)
for i, schedule in enumerate(decision_variables[30:31]):
    x9.append(schedule)
for i, schedule in enumerate(decision_variables[39:40]):
    x9.append(schedule)

    
for i in x9:
    Nurse9 += i
prob += (Nurse9 <= 2)        
    

#for X10
for i, schedule in enumerate(decision_variables[17:18]):
    x10.append(schedule)
for i, schedule in enumerate(decision_variables[24:25]):
    x10.append(schedule)
for i, schedule in enumerate(decision_variables[31:32]):
    x10.append(schedule)   
for i, schedule in enumerate(decision_variables[40:41]):
    x10.append(schedule)      

for i in x10:
    Nurse10 += i
prob += (Nurse10 <= 2)    
    
    
#for X11
for i, schedule in enumerate(decision_variables[7:8]):
    x11.append(schedule)
for i, schedule in enumerate(decision_variables[25:26]):
    x11.append(schedule)
for i, schedule in enumerate(decision_variables[41:42]):
    x11.append(schedule)
    
for i in x11:
    Nurse11 += i
prob += (Nurse11 <= 2)       
      
    
#for X12
for i, schedule in enumerate(decision_variables[8:9]):
    x12.append(schedule)
for i, schedule in enumerate(decision_variables[18:19]):
    x12.append(schedule)
for i, schedule in enumerate(decision_variables[32:33]):
    x12.append(schedule)
for i, schedule in enumerate(decision_variables[42:43]):
    x12.append(schedule)     
    
for i in x12:
    Nurse12 += i
prob += (Nurse12 <= 2)    



In [20]:
#Limit nurses per shift

Nurses_per_shift = 3
Mon_Nurses = ""
Tue_Nurse = ""
Wed_Nurse = ""
Thu_Nurse = ""
Fri_Nurse = ""

for i in decision_variables[0:9]:
    formula = i
    Mon_Nurses += formula

prob += (Mon_Nurses == Nurses_per_shift)

for i in decision_variables[9:19]:
    formula = i
    Tue_Nurse += formula

prob += (Tue_Nurse == Nurses_per_shift)

for i in decision_variables[19:26]:
    formula = i
    Wed_Nurse += formula

prob += (Wed_Nurse == Nurses_per_shift)


for i in decision_variables[26:33]:
    formula = i
    Thu_Nurse += formula

prob += (Thu_Nurse == Nurses_per_shift)

for i in decision_variables[33:43]:
    formula = i
    Fri_Nurse += formula

prob += (Fri_Nurse == Nurses_per_shift)


In [21]:
print prob
prob.writeLP("ScheduleNurses.lp" )

ScheduleNurse:
MINIMIZE
18.0*x0.0MondayAvail + 18.0*x0.0ThursdayAvail + 18.0*x0.0TuesdayAvail + 18.0*x0.0WednesdayAvail + 180.0*x1.0FridayAvail + 180.0*x1.0MondayAvail + 180.0*x1.0ThursdayAvail + 180.0*x1.0TuesdayAvail + 180.0*x1.0WednesdayAvail + 63.0*x10.0FridayAvail + 63.0*x10.0ThursdayAvail + 63.0*x10.0TuesdayAvail + 63.0*x10.0WednesdayAvail + 63.0*x11.0FridayAvail + 63.0*x11.0MondayAvail + 63.0*x11.0WednesdayAvail + 132.0*x12.0FridayAvail + 132.0*x12.0MondayAvail + 132.0*x12.0ThursdayAvail + 132.0*x12.0TuesdayAvail + 95.0*x2.0MondayAvail + 95.0*x2.0TuesdayAvail + 20.0*x3.0FridayAvail + 20.0*x3.0MondayAvail + 20.0*x3.0WednesdayAvail + 20.0*x4.0FridayAvail + 20.0*x4.0ThursdayAvail + 20.0*x4.0TuesdayAvail + 20.0*x5.0FridayAvail + 20.0*x5.0TuesdayAvail + 40.0*x6.0FridayAvail + 40.0*x6.0MondayAvail + 40.0*x6.0WednesdayAvail + 60.0*x7.0MondayAvail + 60.0*x7.0TuesdayAvail + 140.0*x8.0FridayAvail + 140.0*x8.0ThursdayAvail + 140.0*x8.0TuesdayAvail + 140.0*x8.0WednesdayAvail + 200.0*x9.0Fri

In [22]:
optimization_result = prob.solve()
assert optimization_result == pulp.LpStatusOptimal

In [23]:
print(LpStatus[prob.status])

Optimal


In [24]:
print(value(prob.objective))

545.0


In [25]:
for v in prob.variables():
	print(v.name, "=", v.varValue)

('x0.0MondayAvail', '=', 0.0)
('x0.0ThursdayAvail', '=', 1.0)
('x0.0TuesdayAvail', '=', 0.0)
('x0.0WednesdayAvail', '=', 1.0)
('x1.0FridayAvail', '=', 0.0)
('x1.0MondayAvail', '=', 0.0)
('x1.0ThursdayAvail', '=', 0.0)
('x1.0TuesdayAvail', '=', 0.0)
('x1.0WednesdayAvail', '=', 0.0)
('x10.0FridayAvail', '=', 0.0)
('x10.0ThursdayAvail', '=', 1.0)
('x10.0TuesdayAvail', '=', 0.0)
('x10.0WednesdayAvail', '=', 1.0)
('x11.0FridayAvail', '=', 1.0)
('x11.0MondayAvail', '=', 0.0)
('x11.0WednesdayAvail', '=', 0.0)
('x12.0FridayAvail', '=', 0.0)
('x12.0MondayAvail', '=', 0.0)
('x12.0ThursdayAvail', '=', 0.0)
('x12.0TuesdayAvail', '=', 0.0)
('x2.0MondayAvail', '=', 0.0)
('x2.0TuesdayAvail', '=', 0.0)
('x3.0FridayAvail', '=', 0.0)
('x3.0MondayAvail', '=', 1.0)
('x3.0WednesdayAvail', '=', 1.0)
('x4.0FridayAvail', '=', 0.0)
('x4.0ThursdayAvail', '=', 1.0)
('x4.0TuesdayAvail', '=', 1.0)
('x5.0FridayAvail', '=', 1.0)
('x5.0TuesdayAvail', '=', 1.0)
('x6.0FridayAvail', '=', 1.0)
('x6.0MondayAvail', '=', 1.

In [26]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)
    
df2 = pd.DataFrame({'variable':variable_name, 'value': variable_value})
df2 = df2.loc[df2['value']==1.0]


df2['EmpID'] = df2.variable.str.extract(r'(\d+)', expand=True)
df2['EmpID'] = str('x')+ df2.EmpID

df2['Day'] = df2.variable.str.replace('\d+', '')
df2['Day'] = df2.Day.str.replace('x', '')
df2['Day'] = df2.Day.str.replace('Avail', '')

df2.sort_values(by=['Day'])

print (df2)



    value             variable EmpID         Day
1     1.0    x0.0ThursdayAvail    x0   .Thursday
3     1.0   x0.0WednesdayAvail    x0  .Wednesday
10    1.0   x10.0ThursdayAvail   x10   .Thursday
12    1.0  x10.0WednesdayAvail   x10  .Wednesday
13    1.0     x11.0FridayAvail   x11     .Friday
23    1.0      x3.0MondayAvail    x3     .Monday
24    1.0   x3.0WednesdayAvail    x3  .Wednesday
26    1.0    x4.0ThursdayAvail    x4   .Thursday
27    1.0     x4.0TuesdayAvail    x4    .Tuesday
28    1.0      x5.0FridayAvail    x5     .Friday
29    1.0     x5.0TuesdayAvail    x5    .Tuesday
30    1.0      x6.0FridayAvail    x6     .Friday
31    1.0      x6.0MondayAvail    x6     .Monday
33    1.0      x7.0MondayAvail    x7     .Monday
34    1.0     x7.0TuesdayAvail    x7    .Tuesday


In [27]:
fields = ['Nurse']
dfNurses = pd.read_excel(r'C:\Users\jthekkel\Documents\Optimization\NurseOptimizationDetail.xlsx',sheet_name='Data',usecols=fields)

dfNurses['EmpID'] = dfNurses.reset_index().index
dfNurses['EmpID'] = str('x')+ dfNurses.EmpID.map(str)

print(dfNurses)

        Nurse EmpID
0       Jacob    x0
1     Katrina    x1
2       Daisy    x2
3       Tracy    x3
4       Sally    x4
5      Stella    x5
6      Marcus    x6
7   Christina    x7
8        Mark    x8
9       Janet    x9
10       Nina   x10
11      Brian   x11
12    Richard   x12


In [28]:
result = pd.merge(df2,dfNurses, how = 'inner', on = 'EmpID')
result = result.drop(columns=['value','variable','EmpID'])
result.sort_values('Day')

,Day,Nurse
4,.Friday,Brian
9,.Friday,Stella
11,.Friday,Marcus
5,.Monday,Tracy
12,.Monday,Marcus
13,.Monday,Christina
0,.Thursday,Jacob
2,.Thursday,Nina
7,.Thursday,Sally
8,.Tuesday,Sally
